<a href="https://colab.research.google.com/github/ahmed-sala/NLP-Assignment/blob/main/20210064_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional



In [47]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
data['cleaned'] = data['label'].astype(str).apply(preprocess_text)
texts = data['cleaned']
labels = data['sentiment']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)


In [49]:
max_words = 10000
max_len = 200
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['cleaned'])


In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [51]:
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

y_pred_lr = logistic_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression")
print(f"Accuracy: {accuracy_lr:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))

Logistic Regression
Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.86      0.86      4961
    positive       0.86      0.88      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [53]:
data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

texts = data['label'].astype(str)
labels = data['sentiment']

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

X_train, X_test, y_train, y_test = train_test_split(
    X, labels_encoded, test_size=0.2, random_state=42
)

In [56]:


model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Bidirectional(LSTM(128)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)

loss, accuracy_rnn = model.evaluate(X_test, y_test)
print("\nSimple RNN Model")
print(f"Test Accuracy: {accuracy_rnn * 100:.2f}%")


Epoch 1/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.6844 - loss: 0.5680 - val_accuracy: 0.8480 - val_loss: 0.3664
Epoch 2/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.8814 - loss: 0.2954 - val_accuracy: 0.8708 - val_loss: 0.3079
Epoch 3/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.9149 - loss: 0.2272 - val_accuracy: 0.8648 - val_loss: 0.3237
Epoch 4/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.9441 - loss: 0.1563 - val_accuracy: 0.8725 - val_loss: 0.3564
Epoch 5/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.9674 - loss: 0.0999 - val_accuracy: 0.8605 - val_loss: 0.4185
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8608 - loss: 0.4205

Simple RNN Model
Test Accuracy: 86.17%
